In [1]:
# register raw data files using Caiman

from registration import initial_registration

# Open the channel 1 data in ImageJ/FIJI.
# Make a maximum-value z-projection of ch1 data.
# Use the brightness and contrast table to check for a threshold value for the data.
# Any frames of Ch1 that have pixel values higher than this threshold are removed from the data before motion correction.'
# A good threshold will be lower than the streak values and higher than all the values for true data.
# Picking too low of a value (especially with a soma within view) will result in an empty Ch1 stack.

ch1_thresh = input("Ch1 upper threshold (default is 70):")
if ch1_thresh == '':
    ch1_thresh = 70
else:
    ch1_thresh = int(ch1_thresh)

wk_dir = '/Users/benjaminscholl/Documents/autophagy_tal/file_22'
initial_registration(wk_dir, ch1_thresh)


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
Default temporary dir /Users/benjaminscholl/caiman_data/temp does not exist, creating
100%|██████████| 1/1 [00:00<00:00, 22.49it/s]
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
100%|██████████| 1/1 [00:00<00:00, 23.59it/s]
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
100%|██████████| 1/1 [00:00<00:00, 22.26it/s]
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_l

In [ ]:
# Check quality of registration in ImageJ/FIJI
# If quality is good, draw segmented line ROIs for dendrites/axons in Ch2 nonrigid data.
# These are the the regions that will be tracked.

# Create kymographs for tracking particles.
# One set of kymographs will be created for each ROI drawn

kymo_width = 9 # how many pixels wide to include around dendrite/axon
which_kymo = 1 # 1 means use rigid reg, 2 for nonrigid

import os
import sys
sys.path.append('/Users/benjaminscholl/Documents/autophagy_tal/autophagy_code') # change to wherever autophagy_tal is located
import tifffile
import roifile
from create_kymograph import create_tracking_kymo, create_kymograph, create_kymo_movie
from glob import glob

print(f"Making kymos for: {wk_dir}")

if which_kymo == 1:
    ch1 = tifffile.imread('ch1_rigid_registered.tif')
    ch2 = tifffile.imread('ch2_rigid_registered.tif')
elif which_kymo == 2:
    ch1 = tifffile.imread('ch1_nonrigid_registered.tif')
    ch2 = tifffile.imread('ch2_nonrigid_registered.tif')
single_roifile = glob('*.roi')
if 'RoiSet.zip' in os.listdir():
    roi_list = roifile.roiread('RoiSet.zip')
elif any(single_roifile):
    roi_list = roifile.roiread(single_roifile[0])
    roi_list = [roi_list]
for i in range(len(roi_list)):
    roi_coords = roi_list[i].coordinates()
    ch1_kymo, _ = create_kymograph(ch1, roi_coords, kymo_width)
    ch2_kymo, ch2_kymo_movie = create_kymograph(ch2, roi_coords, kymo_width)
    ch2_tracking_kymo = create_tracking_kymo(ch2_kymo, roi_coords)

    if not os.path.exists('kymographs'):
        os.makedirs('kymographs')
    
    tifffile.imwrite(os.path.join('kymographs',f'ch1_kymo_roi_{i}.tif'), ch1_kymo)
    tifffile.imwrite(os.path.join('kymographs' ,f'ch2_kymo_roi_{i}.tif'), ch2_kymo)
    tifffile.imwrite(os.path.join('kymographs' ,f'ch2_tracking_kymo_roi_{i}.tif'), ch2_tracking_kymo)
    print(ch2_kymo_movie.shape)
    create_kymo_movie(ch2_kymo_movie, f'ch2_kymo_mov_roi_{i}.mp4')

Making kymos for: /Users/benjaminscholl/Documents/autophagy_tal/file_22
(9, 119, 200)
Kymograph video saved as ch2_kymo_mov_roi_0.mp4
(9, 31, 200)
Kymograph video saved as ch2_kymo_mov_roi_1.mp4
(9, 27, 200)
Kymograph video saved as ch2_kymo_mov_roi_2.mp4
(9, 34, 200)
Kymograph video saved as ch2_kymo_mov_roi_3.mp4
(9, 22, 200)
Kymograph video saved as ch2_kymo_mov_roi_4.mp4
(9, 31, 200)
Kymograph video saved as ch2_kymo_mov_roi_5.mp4
(9, 43, 200)
Kymograph video saved as ch2_kymo_mov_roi_6.mp4


In [ ]:
# open each kymograph in ImageJ/FIJI
# Use the segmented line tool to draw a line along a 'tracking_kymo_roi'
# draw 

